# Advanced Pandas 4

In [ ]:
import pandas as pd

# 1795. Rearrange Products Table

**Difficulty:** Easy  
**Topics:** SQL, Pandas

## Table: Products

| Column Name | Type    |
|-------------|---------|
| product_id  | int     |
| store1      | int     |
| store2      | int     |
| store3      | int     |

- `product_id` is the primary key (column with unique values) for this table.
- Each row in this table indicates the product's price in 3 different stores: `store1`, `store2`, and `store3`.
- If the product is not available in a store, the price will be `null` in that store's column.

---

## Problem Statement

Write a solution to rearrange the **Products** table so that each row has `(product_id, store, price)`. If a product is not available in a store, do **not** include a row with that `product_id` and `store` combination in the result table.

Return the result table in any order.

---

## Example

### Input: Products Table

| product_id | store1 | store2 | store3 |
|------------|--------|--------|--------|
| 0          | 95     | 100    | 105    |
| 1          | 70     | null   | 80     |

---

### Output

| product_id | store  | price |
|------------|--------|-------|
| 0          | store1 | 95    |
| 0          | store2 | 100   |
| 0          | store3 | 105   |
| 1          | store1 | 70    |
| 1          | store3 | 80    |

---

## Explanation

- **Product 0** is available in all three stores with prices:
  - `store1`: 95, `store2`: 100, `store3`: 105.
- **Product 1** is available in:
  - `store1`: 70 and `store3`: 80.
  - It is not available in `store2`, so no row is included for this combination.


In [ ]:
# melt() is useful to convert a DataFrame from wide format to long format
'''
pd.melt(
    frame,                 # The DataFrame to melt
    id_vars=None,          # Columns to keep as they are (identifier columns)
    value_vars=None,       # Columns to unpivot
    var_name=None,         # Name of the new column for original column names
    value_name="value"     # Name of the new column for original column values
)
'''
# so store1, store2, store3 will be merged into var_name = 'store' 

In [ ]:
def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:

    table = pd.melt(frame=products,id_vars=['product_id'],value_vars={'store1', 'store2','store3'},var_name='store',value_name='price')
    table = table.dropna(subset=['price'])
    return table

In [ ]:
# another solution 

def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:
    return pd.melt(products, id_vars = 'product_id', var_name = 'store', value_name = 'price').dropna()

### 1907. Count Salary Categories

**Difficulty**: Medium  
**Topics**: SQL, Pandas  

---

### **Table: Accounts**

| Column Name | Type |
|-------------|------|
| account_id  | int  |
| income      | int  |

- `account_id` is the primary key (column with unique values) for this table.  
- Each row contains information about the monthly income for one bank account.

---

### **Problem Statement**

Write a solution to calculate the number of bank accounts for each salary category. The salary categories are defined as follows:  

1. **Low Salary**: All salaries strictly less than \$20,000.  
2. **Average Salary**: All salaries in the inclusive range [\$20,000, \$50,000].  
3. **High Salary**: All salaries strictly greater than \$50,000.

The result table must contain all three categories. If there are no accounts in a category, return `0`.  

The result table should have the following columns:  

| Column Name      | Description                       |
|------------------|-----------------------------------|
| category         | The salary category ("Low Salary", "Average Salary", or "High Salary"). |
| accounts_count   | The number of bank accounts in the respective category.                |

---

### **Example**

#### **Input**

`Accounts` table:

| account_id | income |
|------------|--------|
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |

#### **Output**

| category       | accounts_count |
|----------------|----------------|
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |

#### **Explanation**:

- **Low Salary**: Account 2.  
- **Average Salary**: No accounts fall into this range.  
- **High Salary**: Accounts 3, 6, and 8.  

---

### **Constraints**
- Return the result table in any order.

In [ ]:
# wrong solution
def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    conditions = [
        accounts['income'] < 20000,
        (accounts['income'] >= 20000) & (accounts['income'] <= 50000),
        accounts['income'] > 50000
    ]
    categories = ["Low Salary", "Average Salary", "High Salary"]
    
    accounts['category'] = pd.cut(accounts['income'], 
                                  bins=[-float('inf'), 20000, 50000, float('inf')], 
                                  labels=categories, 
                                  right=False)
    category_counts = accounts['category'].value_counts(sort=False)

    result = pd.DataFrame({
        "category": categories,
        "accounts_count": [category_counts.get(cat, 0) for cat in categories]
    })
    
    return result

In [ ]:
# Correct solution from ClaudeAI

def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    # Define the categorization function
    def numAcc(income):
        if income < 20000:
            return "Low Salary"
        elif 20000 <= income <= 50000:
            return "Average Salary"
        else:
            return "High Salary"
    
    accounts['category'] = accounts['income'].apply(numAcc)
    
    category_counts = accounts['category'].value_counts()
    
    all_categories = ["Low Salary", "Average Salary", "High Salary"]
    result = pd.DataFrame({
        "category": all_categories,
        "accounts_count": [category_counts.get(cat, 0) for cat in all_categories]
    })
    
    return result


# 1741. Find Total Time Spent by Each Employee

**Difficulty:** Easy  
**Topics:** SQL, Pandas

---

## Table: Employees

| Column Name | Type |
|-------------|------|
| emp_id      | int  |
| event_day   | date |
| in_time     | int  |
| out_time    | int  |

- `(emp_id, event_day, in_time)` is the primary key (unique combination of these columns).
- The table records employees' entry and exit times at the office.
  - `event_day`: The day on which the event occurred.
  - `in_time`: The minute of the day when the employee entered the office.
  - `out_time`: The minute of the day when the employee left the office.
- Constraints:
  - `1 ≤ in_time < out_time ≤ 1440`.
  - No two events for the same employee on the same day overlap.

---

## Problem Statement

Write a solution to calculate the **total time (in minutes)** spent by each employee on each day in the office. 

### Notes:
- An employee may have multiple entries and exits on the same day.
- The time spent in the office for a single entry is calculated as `out_time - in_time`.

### Return:
The result table should have the following columns:
- `day`: The `event_day` when the employee was present.
- `emp_id`: The ID of the employee.
- `total_time`: The total minutes the employee spent in the office on that day.

The result can be returned in any order.

---

## Example

### Input:
**Employees Table**:
| emp_id | event_day  | in_time | out_time |
|--------|------------|---------|----------|
| 1      | 2020-11-28 | 4       | 32       |
| 1      | 2020-11-28 | 55      | 200      |
| 1      | 2020-12-03 | 1       | 42       |
| 2      | 2020-11-28 | 3       | 33       |
| 2      | 2020-12-09 | 47      | 74       |

### Output:
| day        | emp_id | total_time |
|------------|--------|------------|
| 2020-11-28 | 1      | 173        |
| 2020-11-28 | 2      | 30         |
| 2020-12-03 | 1      | 41         |
| 2020-12-09 | 2      | 27         |

### Explanation:
- **Employee 1**:
  - On `2020-11-28`, they have two events:
    - `(32 - 4) = 28` and `(200 - 55) = 145`.
    - Total for the day: `28 + 145 = 173`.
  - On `2020-12-03`, they have one event:
    - `(42 - 1) = 41`.
- **Employee 2**:
  - On `2020-11-28`, they have one event:
    - `(33 - 3) = 30`.
  - On `2020-12-09`, they have one event:
    - `(74 - 47) = 27`.

---

## Constraints
- The result must include all `event_day` and `emp_id` combinations that appear in the input.
- The result can be returned in any order.


In [ ]:
def total_time(employees: pd.DataFrame) -> pd.DataFrame:
    employees['total_time'] = employees['out_time'] - employees['in_time']
    employees = employees.drop(columns=['in_time', 'out_time'])
    employees = employees.rename(columns={'event_day': 'day'})
    result = employees.groupby(['day', 'emp_id'], as_index=False)['total_time'].sum()
    return result[['day', 'emp_id', 'total_time']]


## 511. Game Play Analysis I

**Easy**

### Topics:
- Companies

### SQL Schema
Table: Activity

| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

**(player_id, event_date) is the primary key (combination of columns with unique values) of this table.**

This table shows the activity of players of some games. Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.

### Problem:

Write a solution to find the first login date for each player.

Return the result table in any order.

### Example 1:

#### Input:
Activity table:

| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-05-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |

#### Output:
| player_id | first_login |
|-----------|-------------|
| 1         | 2016-03-01  |
| 2         | 2017-06-25  |
| 3         | 2016-03-02  |
